興味があるのは一回の更新にどれくらい時間がかかるか  

データセットはMNIST  
全結合ネットワーク



In [37]:
l = [1e4, 5 * 1e4, 1e5, 5 * 1e5, 1e6, 5 * 1e6, 1e7]

result = {}

for v in l:
    print("v: ", v)
    result[v] = (-1, -1, -1, -1)
    min_diff = 1e10
    for i in range(1, 601):
        for j in range(i, 1001):
            k = int((v - 784 * i - i * j) / (j + 10))
            if k > 600 or k < 0:
                continue
            tmp = 784 * i + i * j + j * k + 10 * k
            diff = abs(v - tmp)
            if diff < min_diff:
                result[v] = (tmp, i, j, k)
                min_diff = diff

print(result)

v:  10000.0
v: 50000.0
v: 100000.0
v:  500000.0
v:  1000000.0
v:  5000000.0
v:  10000000.0
{10000.0: (10000, 1, 649, 13), 50000.0: (50000, 1, 141, 325), 100000.0: (100000, 2, 292, 324), 500000.0: (500000, 9, 843, 569), 1000000.0: (1000000, 234, 996, 580), 5000000.0: (-1, -1, -1, -1), 10000000.0: (-1, -1, -1, -1)}


In [38]:
import os
import sys
sys.path.append(os.pardir)
import numpy as np
import matplotlib.pyplot as plt
from collections import OrderedDict

In [39]:
from dataset.mnist import load_mnist
from common.multi_layer_net import MultiLayerNet
from common.layers import Affine, Relu, SoftmaxWithLoss

In [40]:
class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        # 重みの初期化
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size) 
        self.params['b2'] = np.zeros(output_size)

        # レイヤの生成
        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])

        self.lastLayer = SoftmaxWithLoss()

    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)

        return x

    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t)

    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1 :
            t = np.argmax(t, axis=1)

        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy

    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)

        grad = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])

        return grads

    def gradient(self, x, t):
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.lastLayer.backward(dout)

        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        grads = {}
        grads['W1'] = self.layers['Affine1'].dW
        grads['b1'] = self.layers['Affine1'].db
        grads['W2'] = self.layers['Affine2'].dW
        grads['b2'] = self.layers['Affine2'].db

        return grads

In [41]:
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

# network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)
network = MultiLayerNet(input_size=784, hidden_size_list=[50, 50], output_size=10)

networks = {
    '1e3': MultiLayerNet(input_size=784, hidden_size_list=[128, ], output_size=10),
    '1e4': MultiLayerNet(input_size=784, hidden_size_list=[128, ], output_size=10),
    '1e5': MultiLayerNet(input_size=784, hidden_size_list=[128, ], output_size=10),
    '1e6': MultiLayerNet(input_size=784, hidden_size_list=[128, ], output_size=10),
    '1e7': MultiLayerNet(input_size=784, hidden_size_list=[128, ], output_size=10),
    '1e8': MultiLayerNet(input_size=784, hidden_size_list=[128, ], output_size=10),
    '1e9': MultiLayerNet(input_size=784, hidden_size_list=[128, ], output_size=10),
    '1e10': MultiLayerNet(input_size=784, hidden_size_list=[128, ], output_size=10),
}


iters_num = 1000 # average
train_size = x_train.shape[0]
batch_size = 64
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []


In [42]:
def plot_graph():
    markers = {'train': 'o', 'test': 's'}
    x = np.arange(len(train_acc_list))
    plt.plot(x, train_acc_list, label='train acc')
    plt.plot(x, test_acc_list, label='test acc', linestyle='--')
    plt.xlabel("epochs")
    plt.ylabel("accuracy")
    plt.ylim(0, 1.0)
    plt.legend(loc='lower right')
    plt.show()

In [43]:
for i in range(iters_num + 1):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]

    grad = network.gradient(x_batch, t_batch)
    # grad = network.numerical_gradient(x_batch, t_batch)

    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]

    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)

    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print("epoch: {:>5} | train acc: {:.5f}, test acc: {:.5f}".format(i, train_acc, test_acc))

plot_graph()

epoch:     0 | train acc: 0.13737, test acc: 0.12800


KeyboardInterrupt: 